In [1]:
import sys
sys.path.append('..')

from scripts.global_funcs import load_data_config
from glob import glob
import os
os.environ["TF_MEMORY_ALLOCATION"] = "0.7"  # fraction of free memory
os.environ["TF_VISIBLE_DEVICE"] = "0"
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
from nvtabular.loader.tensorflow import KerasSequenceLoader, KerasSequenceValidater
import tensorflow as tf
import nvtabular as nvt
import dask_cudf

# import rmm
# from nvtabular.utils import device_mem_size
# import shutil
# import pathlib
from dask_cuda import LocalCUDACluster
from dask.distributed import Client
import pandas as pd

2022-05-23 07:53:52.571981: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-23 07:53:52.572426: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-23 07:53:52.572580: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [2]:
print("loading configs...")
configs = load_data_config()

loading configs...


In [3]:
print("setting variables...")
output_dir = configs['output_dir']
project_name = configs['project_name']
input_col_name = configs['input_col_name']
label_col_name = configs['label_col_name']
random_seed = configs['random_seed']
data_splits = configs['data_splits']
base_col_names = configs['base_col_names']

setting variables...


In [4]:
base_in_pathname = f"{output_dir}/{project_name}/nvtab"

In [5]:
# # define some information about where to get our data
# dask_workdir = pathlib.Path(base_in_pathname, "dask", "workdir")
# stats_path = pathlib.Path(base_in_pathname, "dask", "stats")

# # Make sure we have a clean worker space for Dask
# if pathlib.Path.is_dir(dask_workdir):
#     shutil.rmtree(dask_workdir)
# dask_workdir.mkdir(parents=True)

# # Make sure we have a clean stats space for Dask
# if pathlib.Path.is_dir(stats_path):
#     shutil.rmtree(stats_path)
# stats_path.mkdir(parents=True)

# # Get device memory capacity
# capacity = device_mem_size(kind="total")

# # Deploy a Single-Machine Multi-GPU Cluster
# protocol = "tcp"  # "tcp" or "ucx"
# visible_devices = "0"  # Delect devices to place workers
# device_spill_frac = 0.2  # Spill GPU-Worker memory to host at this limit.
# # Reduce if spilling fails to prevent
# # device memory errors.
# cluster = None  # (Optional) Specify existing scheduler port
# if cluster is None:
#     cluster = LocalCUDACluster(
#         protocol=protocol,
#         CUDA_VISIBLE_DEVICES=visible_devices,
#         local_directory=dask_workdir,
#         device_memory_limit=capacity * device_spill_frac,
#         rmm_pool_size='1GB'
#     )

# # Create the distributed client
# client = Client(cluster)
# client

# # # Initialize RMM pool on ALL workers
# # def _rmm_pool():
# #     rmm.reinitialize(
# #         pool_allocator=True,
# #         initial_pool_size=None,  # Use default size
# #     )


# # client.run(_rmm_pool)

In [6]:
cluster = LocalCUDACluster()
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:45209 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 1 Cores: 1 Memory: 31.21 GiB


In [7]:
TRAIN_PATHS = sorted(glob(f"{base_in_pathname}/train/*.parquet"))
VAL_PATHS = sorted(glob(f"{base_in_pathname}/val/*.parquet"))
TEST_PATHS = sorted(glob(f"{base_in_pathname}/test/*.parquet"))


In [8]:
TEST_PATHS

['/media/jcosme/Data/full_mer_1/nvtab/test/part_0.parquet']

In [9]:
BATCH_SIZE = 16384
LEARNING_RATE = 0.001
EPOCHS = 50

In [10]:
# workflow = nvt.Workflow.load(f"{base_in_pathname}/workflow")

In [11]:
# dask_cudf.read_parquet(f"{base_in_pathname}/val").head()

In [12]:
# feed them to our datasets
train_dataset = KerasSequenceLoader(
    nvt.Dataset(TRAIN_PATHS, part_size="100MB"), # you could also use a glob pattern
    # TRAIN_PATHS,
    # cat_names=[input_col_name],
    batch_size=BATCH_SIZE,
    label_names=[label_col_name],
    shuffle=True,
    buffer_size=0.001,  # amount of data, as a fraction of GPU memory, to load at once,
    device=0,
    parts_per_chunk=1,
    engine="parquet",
)



/home/jcosme/miniconda3/envs/tf/lib/python3.7/site-packages/merlin/io/parquet.py:323: UserWarning: Row group memory size (132660004) (bytes) of parquet file is bigger than requested part_size (100000000) for the NVTabular dataset.A row group memory size of 128 MB is generally recommended. You can find info on how to set the row group size of parquet files in https://nvidia-merlin.github.io/NVTabular/main/resources/troubleshooting.html#setting-the-row-group-size-for-the-parquet-files
  f"Row group memory size ({rg_byte_size_0}) (bytes) of parquet file is bigger"


In [13]:
valid_dataset = KerasSequenceLoader(
    nvt.Dataset(VAL_PATHS, part_size="100MB"),   # you could also use a glob pattern
    # VAL_PATHS,
    # workflow.transform(cur_dataset),
    # cat_names=[input_col_name],
    batch_size=BATCH_SIZE,
    label_names=[label_col_name],
    shuffle=False,
    buffer_size=0.001,  # amount of data, as a fraction of GPU memory, to load at once,
    device=0,
    parts_per_chunk=1,
    engine="parquet",
)

In [14]:
# test_dataset = KerasSequenceLoader(
#     nvt.Dataset(TEST_PATHS, part_size="100MB"),   # you could also use a glob pattern
#     # TEST_PATHS,
#     # cat_names=[input_col_name],
#     batch_size=BATCH_SIZE,
#     label_names=[label_col_name],
#     shuffle=False,
#     buffer_size=0.06,  # amount of data, as a fraction of GPU memory, to load at once
#     engine="parquet",
#     parts_per_chunk=1,
#     device=0
# )

In [15]:
# valid_dataset

In [16]:
# batch = next(iter(test_dataset))

In [17]:
# batch

In [18]:
inputs = {}
inputs[input_col_name] = \
    (tf.keras.Input(name=f"{input_col_name}__values", dtype=tf.int64, shape=(1,)),
     tf.keras.Input(name=f"{input_col_name}__nnzs", dtype=tf.int32, shape=(1,)))

In [19]:
inputs2 = tf.keras.layers.Lambda(lambda x: x['seq'][0])(inputs)
throw_way = tf.keras.layers.Lambda(lambda x: x['seq'][1])(inputs)
shape = [tf.shape(throw_way)[k] for k in range(2)]
inputs2 = tf.reshape(inputs2, [shape[0], 150])
inputs2 = tf.cast(inputs2, tf.float32)
inputs2 = tf.expand_dims(inputs2, 0)
inputs2 = tf.reshape(inputs2, [shape[0], 1, 150])
inputs2 = tf.math.multiply(inputs2, 1/4)
# inputs2 = tf.transpose(inputs2)

# inputs2 = tf.keras.layers.Lambda(lambda x: tf.split(x, 32))(inputs)

2022-05-23 07:54:31.919770: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-23 07:54:31.920574: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-23 07:54:31.920744: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-23 07:54:31.920857: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

In [20]:
n_classes = pd.read_csv(f"{output_dir}/{project_name}/data/unq_labels.csv" ).shape[0]

In [21]:
# input_layer = tf.keras.layers.Input(shape=(1,150,), batch_size= dtype=tf.float32)

preblock_a = tf.keras.layers.Conv1D(filters=64, kernel_size=3, padding='SAME')(inputs2)
preblock_a = tf.keras.layers.BatchNormalization()(preblock_a)
preblock_a = tf.keras.layers.Activation('gelu')(preblock_a)
preblock_a = tf.keras.layers.MaxPool1D(pool_size=2, padding='SAME')(preblock_a)


# block 1
block_1_1_a = tf.keras.layers.Conv1D(filters=64, kernel_size=3, padding='SAME')(preblock_a)
block_1_1_a = tf.keras.layers.BatchNormalization()(block_1_1_a)
block_1_1_a = tf.keras.layers.Activation('gelu')(block_1_1_a)
block_1_1_a = tf.keras.layers.Conv1D(filters=64, kernel_size=3, padding='SAME')(block_1_1_a)
block_1_1_a = tf.keras.layers.BatchNormalization()(block_1_1_a)
block_1_1_a = tf.keras.layers.Concatenate()([block_1_1_a, preblock_a])
block_1_1_a = tf.keras.layers.Activation('gelu')(block_1_1_a)

block_1_2_a = tf.keras.layers.Conv1D(filters=128, kernel_size=3, padding='SAME')(block_1_1_a)
block_1_2_a = tf.keras.layers.BatchNormalization()(block_1_2_a)
block_1_2_a = tf.keras.layers.Activation('gelu')(block_1_2_a)
block_1_2_a = tf.keras.layers.Conv1D(filters=128, kernel_size=3, padding='SAME')(block_1_2_a)
block_1_2_a = tf.keras.layers.BatchNormalization()(block_1_2_a)
block_1_2_a = tf.keras.layers.Concatenate()([block_1_2_a, block_1_1_a])
block_1_2_a = tf.keras.layers.Activation('gelu')(block_1_2_a)

block_1_3_a = tf.keras.layers.Conv1D(filters=256, kernel_size=3, padding='SAME')(block_1_2_a)
block_1_3_a = tf.keras.layers.BatchNormalization()(block_1_3_a)
block_1_3_a = tf.keras.layers.Activation('gelu')(block_1_3_a)
block_1_3_a = tf.keras.layers.Conv1D(filters=256, kernel_size=3, padding='SAME')(block_1_3_a)
block_1_3_a = tf.keras.layers.BatchNormalization()(block_1_3_a)
block_1_3_a = tf.keras.layers.Concatenate()([block_1_3_a, block_1_2_a])
block_1_3_a = tf.keras.layers.Activation('gelu')(block_1_3_a)

block_1_4_a = tf.keras.layers.Conv1D(filters=512, kernel_size=3, padding='SAME')(block_1_3_a)
block_1_4_a = tf.keras.layers.BatchNormalization()(block_1_4_a)
block_1_4_a = tf.keras.layers.Activation('gelu')(block_1_4_a)
block_1_4_a = tf.keras.layers.Conv1D(filters=512, kernel_size=3, padding='SAME')(block_1_4_a)
block_1_4_a = tf.keras.layers.BatchNormalization()(block_1_4_a)
block_1_4_a = tf.keras.layers.Concatenate()([block_1_4_a, block_1_3_a])
block_1_4_a = tf.keras.layers.Activation('gelu')(block_1_4_a)

block_1_4_a = tf.keras.layers.AveragePooling1D(pool_size=2, padding='SAME')(block_1_4_a)


# block 2
block_2_1_a = tf.keras.layers.Conv1D(filters=64, kernel_size=5, padding='SAME')(preblock_a)
block_2_1_a = tf.keras.layers.BatchNormalization()(block_2_1_a)
block_2_1_a = tf.keras.layers.Activation('gelu')(block_2_1_a)
block_2_1_a = tf.keras.layers.Conv1D(filters=64, kernel_size=5, padding='SAME')(block_2_1_a)
block_2_1_a = tf.keras.layers.BatchNormalization()(block_2_1_a)
block_2_1_a = tf.keras.layers.Concatenate()([block_2_1_a, preblock_a])
block_2_1_a = tf.keras.layers.Activation('gelu')(block_2_1_a)

block_2_2_a = tf.keras.layers.Conv1D(filters=128, kernel_size=5, padding='SAME')(block_2_1_a)
block_2_2_a = tf.keras.layers.BatchNormalization()(block_2_2_a)
block_2_2_a = tf.keras.layers.Activation('gelu')(block_2_2_a)
block_2_2_a = tf.keras.layers.Conv1D(filters=128, kernel_size=5, padding='SAME')(block_2_2_a)
block_2_2_a = tf.keras.layers.BatchNormalization()(block_2_2_a)
block_2_2_a = tf.keras.layers.Concatenate()([block_2_2_a, block_2_1_a])
block_2_2_a = tf.keras.layers.Activation('gelu')(block_2_2_a)

block_2_3_a = tf.keras.layers.Conv1D(filters=256, kernel_size=5, padding='SAME')(block_2_2_a)
block_2_3_a = tf.keras.layers.BatchNormalization()(block_2_3_a)
block_2_3_a = tf.keras.layers.Activation('gelu')(block_2_3_a)
block_2_3_a = tf.keras.layers.Conv1D(filters=256, kernel_size=5, padding='SAME')(block_2_3_a)
block_2_3_a = tf.keras.layers.BatchNormalization()(block_2_3_a)
block_2_3_a = tf.keras.layers.Concatenate()([block_2_3_a, block_2_2_a])
block_2_3_a = tf.keras.layers.Activation('gelu')(block_2_3_a)

block_2_4_a = tf.keras.layers.Conv1D(filters=512, kernel_size=5, padding='SAME')(block_2_3_a)
block_2_4_a = tf.keras.layers.BatchNormalization()(block_2_4_a)
block_2_4_a = tf.keras.layers.Activation('gelu')(block_2_4_a)
block_2_4_a = tf.keras.layers.Conv1D(filters=512, kernel_size=5, padding='SAME')(block_2_4_a)
block_2_4_a = tf.keras.layers.BatchNormalization()(block_2_4_a)
block_2_4_a = tf.keras.layers.Concatenate()([block_2_4_a, block_2_3_a])
block_2_4_a = tf.keras.layers.Activation('gelu')(block_2_4_a)

block_2_4_a = tf.keras.layers.AveragePooling1D(pool_size=2, padding='SAME')(block_2_4_a)

# block 3
block_3_1_a = tf.keras.layers.Conv1D(filters=64, kernel_size=7, padding='SAME')(preblock_a)
block_3_1_a = tf.keras.layers.BatchNormalization()(block_3_1_a)
block_3_1_a = tf.keras.layers.Activation('gelu')(block_3_1_a)
block_3_1_a = tf.keras.layers.Conv1D(filters=64, kernel_size=7, padding='SAME')(block_3_1_a)
block_3_1_a = tf.keras.layers.BatchNormalization()(block_3_1_a)
block_3_1_a = tf.keras.layers.Concatenate()([block_3_1_a, preblock_a])
block_3_1_a = tf.keras.layers.Activation('gelu')(block_3_1_a)

block_3_2_a = tf.keras.layers.Conv1D(filters=128, kernel_size=7, padding='SAME')(block_3_1_a)
block_3_2_a = tf.keras.layers.BatchNormalization()(block_3_2_a)
block_3_2_a = tf.keras.layers.Activation('gelu')(block_3_2_a)
block_3_2_a = tf.keras.layers.Conv1D(filters=128, kernel_size=7, padding='SAME')(block_3_2_a)
block_3_2_a = tf.keras.layers.BatchNormalization()(block_3_2_a)
block_3_2_a = tf.keras.layers.Concatenate()([block_3_2_a, block_3_1_a])
block_3_2_a = tf.keras.layers.Activation('gelu')(block_3_2_a)

block_3_3_a = tf.keras.layers.Conv1D(filters=256, kernel_size=7, padding='SAME')(block_3_2_a)
block_3_3_a = tf.keras.layers.BatchNormalization()(block_3_3_a)
block_3_3_a = tf.keras.layers.Activation('gelu')(block_3_3_a)
block_3_3_a = tf.keras.layers.Conv1D(filters=256, kernel_size=7, padding='SAME')(block_3_3_a)
block_3_3_a = tf.keras.layers.BatchNormalization()(block_3_3_a)
block_3_3_a = tf.keras.layers.Concatenate()([block_3_3_a, block_3_2_a])
block_3_3_a = tf.keras.layers.Activation('gelu')(block_3_3_a)

block_3_4_a = tf.keras.layers.Conv1D(filters=512, kernel_size=7, padding='SAME')(block_3_3_a)
block_3_4_a = tf.keras.layers.BatchNormalization()(block_3_4_a)
block_3_4_a = tf.keras.layers.Activation('gelu')(block_3_4_a)
block_3_4_a = tf.keras.layers.Conv1D(filters=512, kernel_size=7, padding='SAME')(block_3_4_a)
block_3_4_a = tf.keras.layers.BatchNormalization()(block_3_4_a)
block_3_4_a = tf.keras.layers.Concatenate()([block_3_4_a, block_3_3_a])
block_3_4_a = tf.keras.layers.Activation('gelu')(block_3_4_a)

block_3_4_a = tf.keras.layers.AveragePooling1D(pool_size=2, padding='SAME')(block_3_4_a)

preblock_b = tf.keras.layers.Conv1D(filters=64, kernel_size=5, padding='SAME')(inputs2)
preblock_b = tf.keras.layers.BatchNormalization()(preblock_b)
preblock_b = tf.keras.layers.Activation('gelu')(preblock_b)
preblock_b = tf.keras.layers.MaxPool1D(pool_size=2, padding='SAME')(preblock_b)


# block 1
block_1_1_b = tf.keras.layers.Conv1D(filters=64, kernel_size=3, padding='SAME')(preblock_b)
block_1_1_b = tf.keras.layers.BatchNormalization()(block_1_1_b)
block_1_1_b = tf.keras.layers.Activation('gelu')(block_1_1_b)
block_1_1_b = tf.keras.layers.Conv1D(filters=64, kernel_size=3, padding='SAME')(block_1_1_b)
block_1_1_b = tf.keras.layers.BatchNormalization()(block_1_1_b)
block_1_1_b = tf.keras.layers.Concatenate()([block_1_1_b, preblock_b])
block_1_1_b = tf.keras.layers.Activation('gelu')(block_1_1_b)

block_1_2_b = tf.keras.layers.Conv1D(filters=128, kernel_size=3, padding='SAME')(block_1_1_b)
block_1_2_b = tf.keras.layers.BatchNormalization()(block_1_2_b)
block_1_2_b = tf.keras.layers.Activation('gelu')(block_1_2_b)
block_1_2_b = tf.keras.layers.Conv1D(filters=128, kernel_size=3, padding='SAME')(block_1_2_b)
block_1_2_b = tf.keras.layers.BatchNormalization()(block_1_2_b)
block_1_2_b = tf.keras.layers.Concatenate()([block_1_2_b, block_1_1_b])
block_1_2_b = tf.keras.layers.Activation('gelu')(block_1_2_b)

block_1_3_b = tf.keras.layers.Conv1D(filters=256, kernel_size=3, padding='SAME')(block_1_2_b)
block_1_3_b = tf.keras.layers.BatchNormalization()(block_1_3_b)
block_1_3_b = tf.keras.layers.Activation('gelu')(block_1_3_b)
block_1_3_b = tf.keras.layers.Conv1D(filters=256, kernel_size=3, padding='SAME')(block_1_3_b)
block_1_3_b = tf.keras.layers.BatchNormalization()(block_1_3_b)
block_1_3_b = tf.keras.layers.Concatenate()([block_1_3_b, block_1_2_b])
block_1_3_b = tf.keras.layers.Activation('gelu')(block_1_3_b)

block_1_4_b = tf.keras.layers.Conv1D(filters=512, kernel_size=3, padding='SAME')(block_1_3_b)
block_1_4_b = tf.keras.layers.BatchNormalization()(block_1_4_b)
block_1_4_b = tf.keras.layers.Activation('gelu')(block_1_4_b)
block_1_4_b = tf.keras.layers.Conv1D(filters=512, kernel_size=3, padding='SAME')(block_1_4_b)
block_1_4_b = tf.keras.layers.BatchNormalization()(block_1_4_b)
block_1_4_b = tf.keras.layers.Concatenate()([block_1_4_b, block_1_3_b])
block_1_4_b = tf.keras.layers.Activation('gelu')(block_1_4_b)

block_1_4_b = tf.keras.layers.AveragePooling1D(pool_size=2, padding='SAME')(block_1_4_b)


# block 2
block_2_1_b = tf.keras.layers.Conv1D(filters=64, kernel_size=5, padding='SAME')(preblock_b)
block_2_1_b = tf.keras.layers.BatchNormalization()(block_2_1_b)
block_2_1_b = tf.keras.layers.Activation('gelu')(block_2_1_b)
block_2_1_b = tf.keras.layers.Conv1D(filters=64, kernel_size=5, padding='SAME')(block_2_1_b)
block_2_1_b = tf.keras.layers.BatchNormalization()(block_2_1_b)
block_2_1_b = tf.keras.layers.Concatenate()([block_2_1_b, preblock_b])
block_2_1_b = tf.keras.layers.Activation('gelu')(block_2_1_b)

block_2_2_b = tf.keras.layers.Conv1D(filters=128, kernel_size=5, padding='SAME')(block_2_1_b)
block_2_2_b = tf.keras.layers.BatchNormalization()(block_2_2_b)
block_2_2_b = tf.keras.layers.Activation('gelu')(block_2_2_b)
block_2_2_b = tf.keras.layers.Conv1D(filters=128, kernel_size=5, padding='SAME')(block_2_2_b)
block_2_2_b = tf.keras.layers.BatchNormalization()(block_2_2_b)
block_2_2_b = tf.keras.layers.Concatenate()([block_2_2_b, block_2_1_b])
block_2_2_b = tf.keras.layers.Activation('gelu')(block_2_2_b)

block_2_3_b = tf.keras.layers.Conv1D(filters=256, kernel_size=5, padding='SAME')(block_2_2_b)
block_2_3_b = tf.keras.layers.BatchNormalization()(block_2_3_b)
block_2_3_b = tf.keras.layers.Activation('gelu')(block_2_3_b)
block_2_3_b = tf.keras.layers.Conv1D(filters=256, kernel_size=5, padding='SAME')(block_2_3_b)
block_2_3_b = tf.keras.layers.BatchNormalization()(block_2_3_b)
block_2_3_b = tf.keras.layers.Concatenate()([block_2_3_b, block_2_2_b])
block_2_3_b = tf.keras.layers.Activation('gelu')(block_2_3_b)

block_2_4_b = tf.keras.layers.Conv1D(filters=512, kernel_size=5, padding='SAME')(block_2_3_b)
block_2_4_b = tf.keras.layers.BatchNormalization()(block_2_4_b)
block_2_4_b = tf.keras.layers.Activation('gelu')(block_2_4_b)
block_2_4_b = tf.keras.layers.Conv1D(filters=512, kernel_size=5, padding='SAME')(block_2_4_b)
block_2_4_b = tf.keras.layers.BatchNormalization()(block_2_4_b)
block_2_4_b = tf.keras.layers.Concatenate()([block_2_4_b, block_2_3_b])
block_2_4_b = tf.keras.layers.Activation('gelu')(block_2_4_b)

block_2_4_b = tf.keras.layers.AveragePooling1D(pool_size=2, padding='SAME')(block_2_4_b)

# block 3
block_3_1_b = tf.keras.layers.Conv1D(filters=64, kernel_size=7, padding='SAME')(preblock_b)
block_3_1_b = tf.keras.layers.BatchNormalization()(block_3_1_b)
block_3_1_b = tf.keras.layers.Activation('gelu')(block_3_1_b)
block_3_1_b = tf.keras.layers.Conv1D(filters=64, kernel_size=7, padding='SAME')(block_3_1_b)
block_3_1_b = tf.keras.layers.BatchNormalization()(block_3_1_b)
block_3_1_b = tf.keras.layers.Concatenate()([block_3_1_b, preblock_b])
block_3_1_b = tf.keras.layers.Activation('gelu')(block_3_1_b)

block_3_2_b = tf.keras.layers.Conv1D(filters=128, kernel_size=7, padding='SAME')(block_3_1_b)
block_3_2_b = tf.keras.layers.BatchNormalization()(block_3_2_b)
block_3_2_b = tf.keras.layers.Activation('gelu')(block_3_2_b)
block_3_2_b = tf.keras.layers.Conv1D(filters=128, kernel_size=7, padding='SAME')(block_3_2_b)
block_3_2_b = tf.keras.layers.BatchNormalization()(block_3_2_b)
block_3_2_b = tf.keras.layers.Concatenate()([block_3_2_b, block_3_1_b])
block_3_2_b = tf.keras.layers.Activation('gelu')(block_3_2_b)

block_3_3_b = tf.keras.layers.Conv1D(filters=256, kernel_size=7, padding='SAME')(block_3_2_b)
block_3_3_b = tf.keras.layers.BatchNormalization()(block_3_3_b)
block_3_3_b = tf.keras.layers.Activation('gelu')(block_3_3_b)
block_3_3_b = tf.keras.layers.Conv1D(filters=256, kernel_size=7, padding='SAME')(block_3_3_b)
block_3_3_b = tf.keras.layers.BatchNormalization()(block_3_3_b)
block_3_3_b = tf.keras.layers.Concatenate()([block_3_3_b, block_3_2_b])
block_3_3_b = tf.keras.layers.Activation('gelu')(block_3_3_b)

block_3_4_b = tf.keras.layers.Conv1D(filters=512, kernel_size=7, padding='SAME')(block_3_3_b)
block_3_4_b = tf.keras.layers.BatchNormalization()(block_3_4_b)
block_3_4_b = tf.keras.layers.Activation('gelu')(block_3_4_b)
block_3_4_b = tf.keras.layers.Conv1D(filters=512, kernel_size=7, padding='SAME')(block_3_4_b)
block_3_4_b = tf.keras.layers.BatchNormalization()(block_3_4_b)
block_3_4_b = tf.keras.layers.Concatenate()([block_3_4_b, block_3_3_b])
block_3_4_b = tf.keras.layers.Activation('gelu')(block_3_4_b)

block_3_4_b = tf.keras.layers.AveragePooling1D(pool_size=2, padding='SAME')(block_3_4_b)

preblock_c = tf.keras.layers.Conv1D(filters=64, kernel_size=7, padding='SAME')(inputs2)
preblock_c = tf.keras.layers.BatchNormalization()(preblock_c)
preblock_c = tf.keras.layers.Activation('gelu')(preblock_c)
preblock_c = tf.keras.layers.MaxPool1D(pool_size=2, padding='SAME')(preblock_c)


# block 1
block_1_1_c = tf.keras.layers.Conv1D(filters=64, kernel_size=3, padding='SAME')(preblock_c)
block_1_1_c = tf.keras.layers.BatchNormalization()(block_1_1_c)
block_1_1_c = tf.keras.layers.Activation('gelu')(block_1_1_c)
block_1_1_c = tf.keras.layers.Conv1D(filters=64, kernel_size=3, padding='SAME')(block_1_1_c)
block_1_1_c = tf.keras.layers.BatchNormalization()(block_1_1_c)
block_1_1_c = tf.keras.layers.Concatenate()([block_1_1_c, preblock_c])
block_1_1_c = tf.keras.layers.Activation('gelu')(block_1_1_c)

block_1_2_c = tf.keras.layers.Conv1D(filters=128, kernel_size=3, padding='SAME')(block_1_1_c)
block_1_2_c = tf.keras.layers.BatchNormalization()(block_1_2_c)
block_1_2_c = tf.keras.layers.Activation('gelu')(block_1_2_c)
block_1_2_c = tf.keras.layers.Conv1D(filters=128, kernel_size=3, padding='SAME')(block_1_2_c)
block_1_2_c = tf.keras.layers.BatchNormalization()(block_1_2_c)
block_1_2_c = tf.keras.layers.Concatenate()([block_1_2_c, block_1_1_c])
block_1_2_c = tf.keras.layers.Activation('gelu')(block_1_2_c)

block_1_3_c = tf.keras.layers.Conv1D(filters=256, kernel_size=3, padding='SAME')(block_1_2_c)
block_1_3_c = tf.keras.layers.BatchNormalization()(block_1_3_c)
block_1_3_c = tf.keras.layers.Activation('gelu')(block_1_3_c)
block_1_3_c = tf.keras.layers.Conv1D(filters=256, kernel_size=3, padding='SAME')(block_1_3_c)
block_1_3_c = tf.keras.layers.BatchNormalization()(block_1_3_c)
block_1_3_c = tf.keras.layers.Concatenate()([block_1_3_c, block_1_2_c])
block_1_3_c = tf.keras.layers.Activation('gelu')(block_1_3_c)

block_1_4_c = tf.keras.layers.Conv1D(filters=512, kernel_size=3, padding='SAME')(block_1_3_c)
block_1_4_c = tf.keras.layers.BatchNormalization()(block_1_4_c)
block_1_4_c = tf.keras.layers.Activation('gelu')(block_1_4_c)
block_1_4_c = tf.keras.layers.Conv1D(filters=512, kernel_size=3, padding='SAME')(block_1_4_c)
block_1_4_c = tf.keras.layers.BatchNormalization()(block_1_4_c)
block_1_4_c = tf.keras.layers.Concatenate()([block_1_4_c, block_1_3_c])
block_1_4_c = tf.keras.layers.Activation('gelu')(block_1_4_c)

block_1_4_c = tf.keras.layers.AveragePooling1D(pool_size=2, padding='SAME')(block_1_4_c)


# block 2
block_2_1_c = tf.keras.layers.Conv1D(filters=64, kernel_size=5, padding='SAME')(preblock_c)
block_2_1_c = tf.keras.layers.BatchNormalization()(block_2_1_c)
block_2_1_c = tf.keras.layers.Activation('gelu')(block_2_1_c)
block_2_1_c = tf.keras.layers.Conv1D(filters=64, kernel_size=5, padding='SAME')(block_2_1_c)
block_2_1_c = tf.keras.layers.BatchNormalization()(block_2_1_c)
block_2_1_c = tf.keras.layers.Concatenate()([block_2_1_c, preblock_c])
block_2_1_c = tf.keras.layers.Activation('gelu')(block_2_1_c)

block_2_2_c = tf.keras.layers.Conv1D(filters=128, kernel_size=5, padding='SAME')(block_2_1_c)
block_2_2_c = tf.keras.layers.BatchNormalization()(block_2_2_c)
block_2_2_c = tf.keras.layers.Activation('gelu')(block_2_2_c)
block_2_2_c = tf.keras.layers.Conv1D(filters=128, kernel_size=5, padding='SAME')(block_2_2_c)
block_2_2_c = tf.keras.layers.BatchNormalization()(block_2_2_c)
block_2_2_c = tf.keras.layers.Concatenate()([block_2_2_c, block_2_1_c])
block_2_2_c = tf.keras.layers.Activation('gelu')(block_2_2_c)

block_2_3_c = tf.keras.layers.Conv1D(filters=256, kernel_size=5, padding='SAME')(block_2_2_c)
block_2_3_c = tf.keras.layers.BatchNormalization()(block_2_3_c)
block_2_3_c = tf.keras.layers.Activation('gelu')(block_2_3_c)
block_2_3_c = tf.keras.layers.Conv1D(filters=256, kernel_size=5, padding='SAME')(block_2_3_c)
block_2_3_c = tf.keras.layers.BatchNormalization()(block_2_3_c)
block_2_3_c = tf.keras.layers.Concatenate()([block_2_3_c, block_2_2_c])
block_2_3_c = tf.keras.layers.Activation('gelu')(block_2_3_c)

block_2_4_c = tf.keras.layers.Conv1D(filters=512, kernel_size=5, padding='SAME')(block_2_3_c)
block_2_4_c = tf.keras.layers.BatchNormalization()(block_2_4_c)
block_2_4_c = tf.keras.layers.Activation('gelu')(block_2_4_c)
block_2_4_c = tf.keras.layers.Conv1D(filters=512, kernel_size=5, padding='SAME')(block_2_4_c)
block_2_4_c = tf.keras.layers.BatchNormalization()(block_2_4_c)
block_2_4_c = tf.keras.layers.Concatenate()([block_2_4_c, block_2_3_c])
block_2_4_c = tf.keras.layers.Activation('gelu')(block_2_4_c)

block_2_4_c = tf.keras.layers.AveragePooling1D(pool_size=2, padding='SAME')(block_2_4_c)

# block 3
block_3_1_c = tf.keras.layers.Conv1D(filters=64, kernel_size=7, padding='SAME')(preblock_c)
block_3_1_c = tf.keras.layers.BatchNormalization()(block_3_1_c)
block_3_1_c = tf.keras.layers.Activation('gelu')(block_3_1_c)
block_3_1_c = tf.keras.layers.Conv1D(filters=64, kernel_size=7, padding='SAME')(block_3_1_c)
block_3_1_c = tf.keras.layers.BatchNormalization()(block_3_1_c)
block_3_1_c = tf.keras.layers.Concatenate()([block_3_1_c, preblock_c])
block_3_1_c = tf.keras.layers.Activation('gelu')(block_3_1_c)

block_3_2_c = tf.keras.layers.Conv1D(filters=128, kernel_size=7, padding='SAME')(block_3_1_c)
block_3_2_c = tf.keras.layers.BatchNormalization()(block_3_2_c)
block_3_2_c = tf.keras.layers.Activation('gelu')(block_3_2_c)
block_3_2_c = tf.keras.layers.Conv1D(filters=128, kernel_size=7, padding='SAME')(block_3_2_c)
block_3_2_c = tf.keras.layers.BatchNormalization()(block_3_2_c)
block_3_2_c = tf.keras.layers.Concatenate()([block_3_2_c, block_3_1_c])
block_3_2_c = tf.keras.layers.Activation('gelu')(block_3_2_c)

block_3_3_c = tf.keras.layers.Conv1D(filters=256, kernel_size=7, padding='SAME')(block_3_2_c)
block_3_3_c = tf.keras.layers.BatchNormalization()(block_3_3_c)
block_3_3_c = tf.keras.layers.Activation('gelu')(block_3_3_c)
block_3_3_c = tf.keras.layers.Conv1D(filters=256, kernel_size=7, padding='SAME')(block_3_3_c)
block_3_3_c = tf.keras.layers.BatchNormalization()(block_3_3_c)
block_3_3_c = tf.keras.layers.Concatenate()([block_3_3_c, block_3_2_c])
block_3_3_c = tf.keras.layers.Activation('gelu')(block_3_3_c)

block_3_4_c = tf.keras.layers.Conv1D(filters=512, kernel_size=7, padding='SAME')(block_3_3_c)
block_3_4_c = tf.keras.layers.BatchNormalization()(block_3_4_c)
block_3_4_c = tf.keras.layers.Activation('gelu')(block_3_4_c)
block_3_4_c = tf.keras.layers.Conv1D(filters=512, kernel_size=7, padding='SAME')(block_3_4_c)
block_3_4_c = tf.keras.layers.BatchNormalization()(block_3_4_c)
block_3_4_c = tf.keras.layers.Concatenate()([block_3_4_c, block_3_3_c])
block_3_4_c = tf.keras.layers.Activation('gelu')(block_3_4_c)

block_3_4_c = tf.keras.layers.AveragePooling1D(pool_size=2, padding='SAME')(block_3_4_c)

output_layer = tf.keras.layers.Concatenate()([block_1_4_a, block_2_4_a, block_3_4_a, block_1_4_b, block_2_4_b, block_3_4_b, block_1_4_c, block_2_4_c, block_3_4_c])

# output_layer = tf.keras.layers.Concatenate()([block_1_4_a, block_2_4_a, block_3_4_a,])
output_layer = tf.keras.layers.Flatten()(output_layer)
output_layer = tf.keras.layers.Dense(n_classes)(output_layer)
output_layer = tf.keras.layers.Activation('tanh')(output_layer)
# output_layer = tf.keras.layers.Dropout(0.8)(output_layer)
output_layer = tf.keras.layers.Dense(n_classes)(output_layer)
output_layer = tf.keras.layers.LeakyReLU(0.2)(output_layer)
# output_layer = tf.keras.layers.Dropout(0.8)(output_layer)
output_layer = tf.keras.layers.Dense(n_classes)(output_layer)
output_layer = tf.keras.layers.Activation('softmax')(output_layer)
# output_layer = tf.keras.layers.Dropout(0.8)(output_layer)
# output_layer = tf.keras.layers.Dense(n_classes)(output_layer)


In [22]:
model = tf.keras.Model(inputs=inputs, outputs=output_layer)

In [23]:
optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE, amsgrad=True)

In [24]:
model.compile(optimizer, 
              tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False), 
              metrics=['sparse_categorical_accuracy'],
             )

In [25]:
# model.summary()

In [26]:
validation_callback = KerasSequenceValidater(valid_dataset)

In [27]:
history = model.fit(
    train_dataset,
    # validation_data = valid_dataset,
    epochs=EPOCHS,
    callbacks=[validation_callback]
)

Epoch 1/50


2022-05-23 07:55:16.486044: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8201
2022-05-23 07:55:17.159022: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-05-23 07:55:18.815988: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


 35/817 [>.............................] - ETA: 9:53 - loss: 4.7590 - sparse_categorical_accuracy: 0.1795

distributed.nanny - WARNING - Restarting worker


KeyboardInterrupt: 

In [27]:
# model.summary()